In [36]:
import json
import sqlite3 as sql
import pathlib as plib
import pandas as pd

In [23]:
import os
print(os.getcwd())

d:\repos\dadbot-py\tests


In [12]:
## Joke file paths
chuck_file = 'chuck_noris_only_list'
dad_file = 'dad_joke_list'

# Raw Joke Path
joke_dir_name = '/dadbot-py/data/_raw_json_scrapes'
suffix = '.json'


In [26]:
db_path = plib.Path('../data/_sqlite/dadbot_db.db')
db_path

WindowsPath('../data/_sqlite/dadbot_db.db')

In [27]:
try:
    db_path.parent.mkdir(parents=True, exist_ok=True)
except Exception as e:
    print(f"Unable to create directory {db_path.parent}. Error: {e}")
else:
    print(f"Directory {db_path.parent} has been created successfully.")

Directory ..\data\_sqlite has been created successfully.


In [29]:
try:
    conn = sql.connect(db_path)
except Exception as e:
    print(f"Unable to connect to the database at {db_path}. Error: {e}")
else:
    print(f"Connected to the database at {db_path} successfully.")
    cursor = conn.cursor()

Connected to the database at ..\data\_sqlite\dadbot_db.db successfully.


In [30]:
# Checking if the table is already exists in the database. 
# Creates if it doesnt exist...

cursor.execute(
    '''
    SELECT 
        name
    FROM
        sqlite_master
    WHERE
        type = 'table'
        AND name = 'jokes'    
    '''
)
if not cursor.fetchone():
    cursor.execute(
        '''
        CREATE TABLE chuck_jokes (
            jokeID TEXT PRIMARY KEY,
            jokeContent TEXT,
            jokeChosenCnt INTEGER,
            jokeUpVoteCnt INTEGER,
            jokeDwnVoteCnt INTEGER
        )
        '''
    )
    conn.commit()


In [33]:
# Reading in jokes from json file (Chuck norris jokes)

chuck_file = plib.Path.cwd().parent / 'data' / '_raw_json_scrapes' / 'chuck_norris_only_list.json'

try:
    with open(chuck_file, 'r') as file:
        jokes = json.load(file)
except Exception as e:
    print(e)
        

In [35]:
# Inserting chuck norris jokes into the database
for chuck_joke in jokes:
    cursor.execute(
        '''
        INSERT INTO chuck_jokes (
            jokeID,
            jokeContent,
            jokeChosenCnt,
            jokeUpVoteCnt, 
            jokeDwnVoteCnt
        )
        VALUES (
            :jokeID,
            :jokeContent,
            :jokeChosenCnt,
            :jokeUpVoteCnt,
            :jokeDwnVoteCnt
        )
        ''', chuck_joke
    )
conn.commit()

In [38]:
test_query = '''SELECT * FROM chuck_jokes ORDER BY RANDOM() LIMIT 2'''

df = pd.read_sql_query(test_query, conn)

In [41]:
df.iloc[1]

jokeID                         018a9f61-4078-7f24-b82c-4e2bbfa13057
jokeContent       When Chuck Norris is in Rome, the Romans must ...
jokeChosenCnt                                                     0
jokeUpVoteCnt                                                     0
jokeDwnVoteCnt                                                    0
Name: 1, dtype: object

In [42]:
conn.close()

In [32]:
plib.Path.cwd().parent / 'data' / '_raw_json_scrapes' / 'chuck_norris_only_list.json'

WindowsPath('d:/repos/dadbot-py/data/_raw_json_scrapes/chuck_norris_only_list.json')